In [ ]:
# 编码器的最后一步输出，作为解码器的初始状态。
# 解码器每一步预测输出，作为下一步输入。
# 有两种停止方式，一是出现指定字符，二是固定长度。
# 为加速训练收敛，增加了教师强制（teacher Forcing）机制。将真实值作为下一步输入，加速收敛。

# 机器翻译

# 解压数据

In [ ]:
import zipfile
import os

def extract_file(root_path,extract_path):
    '''
    解压文件
    '''
    if os.path.exists(extract_path):
        os.makedirs(extract_path)
    with zipfile.ZipFile(root_path,'r') as zip:
        zip.extract(path = extract_path)

root_path = r''
extract_path = r''
extract_file(root_path,extract_path)

# 词汇表类

In [2]:
# 定义开始字符与结束字符号的索引
sos_token = 0
eos_token = 1

class Lang:
    '''词汇表类，用于构建词汇表和管理词汇表
    支持将单词映射为对应的索引，并记录每个单词的出现次数
    '''
    def __init__(self,name):
        self.name = name
        self.word2idx = {}
        self.word2count = {}
        self.idx2word = {0:'sos',1:'eos'}
        self.n_words = 2

    def addSentence(self,sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self,word):
        if word not in self.word2idx:
            self.word2idx[word] = self.n_words
            self.word2count[word] = 1
            self.idx2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
        
lang = Lang('eng')
lang.addSentence('I am a student')
lang.addSentence('I am a teacher')

print(lang.word2count)
print(lang.word2idx)
print(lang.idx2word)
print(lang.n_words)

{'I': 2, 'am': 2, 'a': 2, 'student': 1, 'teacher': 1}
{'I': 2, 'am': 3, 'a': 4, 'student': 5, 'teacher': 6}
{0: 'sos', 1: 'eos', 2: 'I', 3: 'am', 4: 'a', 5: 'student', 6: 'teacher'}
7


# 读取数据集，并创建词汇表

In [4]:
import unicodedata
import re

def UnicodeToAsceli(s):
    '''
    将字符内容正规化
    '''
    s = unicodedata.normalize('NFD',s)
    return ''.join(c for c in s if unicodedata.category(c) != 'Mn')

def normalizeString(s):
    '''
    将字符内容正规化，小写，并去除特殊字符
    '''
    s = UnicodeToAsceli(s.lower().strip())
    # \1 表示引用第一个捕获组，将该组内容前面加一个空格
    s = re.sub(r'([.?!])',r' \1',s)
    # 将英文吗，！？之外的符号替换为空格
    s = re.sub(r'[^a-zA-Z!?]',r' ',s)
    return s.strip()

def readlangs(lang1,lang2,reverse = False):
    '''从文件中读取平行数据集，并创建词汇表
    parameter:
    ------------
    lang1:str
        语言1名称（元语言）
    lang2:ste
        语言2名称（目标语言）
    reverse:bool
        是否颠倒平行数据
    '''
    pairs = []
    with open(r'.\data\eng-fra.txt' , encoding='utf-8') as f:
        for line in f:
            pairs.append([normalizeString(s) for s in line.strip().split('\t')])

    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang,output_lang,pairs

i,o ,pairs = readlangs('eng','fra')

In [6]:
pairs[10:15]

[['wait !', 'attends  !'],
 ['wait !', 'attendez  !'],
 ['i see', 'je comprends'],
 ['i try', 'j essaye'],
 ['i won !', 'j ai gagne  !']]

# 生成数据集

In [12]:
# 设置序列最大长度
max_length = 10

# prefixes 表示英语的前缀
# 在正则处理中，将'都处理为空格，因此，需要将一些内容替换掉
eng_prefixes = ('i am','i m',
                'he is','he s',
                'she is','she s',
                'you are','you r',
                'they are','they re',
                'we are','we re')

def filterPair(p):
    con1 = len(p[0].split(' ')) < max_length
    con2 = len(p[1].split(' ')) < max_length
    con3 = p[1].startswith(eng_prefixes)

    return con1 and con2 and con3

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [13]:
import random

def preparData(lang1,lang2,reverse =False):
    # 读取词汇数据
    input_lang,output_lang,pairs = readlangs(lang1,lang2,reverse)
    print('读取语句数：',len(pairs))
    # 过滤词汇数据
    pairs = filterPairs(pairs)
    print('过滤之后的数据长度：',len(pairs))

    # 将过滤后的词汇添加到词汇表中
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    
    print(input_lang.name,input_lang.n_words)
    print(output_lang.name,output_lang.n_words)

    return input_lang,output_lang,pairs

input_lang,output_lang,pairs = preparData('eng','fra',True)
print(random.choice(pairs))
        

读取语句数： 135842
过滤之后的数据长度： 12684
fra 5192
eng 3419
['je me rejouis de vous voir danser', 'i m looking forward to seeing you dance']


# encoder-decoder

In [17]:
#encoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as fc
from torch.utils.data import Dataset,DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class EncoderRNN(nn.Module):
    def __init__(self,vocab_size,embedding_dim,hidden_size,dropout_p = 0.1):
        '''
        初始化方法
        
        parameter:
        -------------
        vocab_size:int
            词嵌入的数量
        embedding_dim:int
            词嵌入的维度
        hidden_size:int
            RNN的数量
        dropout_p:float
            神经元丢弃比例
        '''
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.gru = nn.GRU(embedding_dim,hidden_size,batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self,x):
        
        # input shape = (batch_size,seq_len)
        # embedding shape = (batch_size,seq_len,embedding_dim)
        embeding_x = self.embedding(x)
        # output shape = (batch_size,seq_len,D*hidden_size)
        # hidden shape = (D*num_layer,batch_size,hidden_size)
        output,hidden = self.gru(embeding_x)
        
        return output,hidden



In [ ]:
# decoder

class Decoder(nn.Module):
    def __init__(self,embedding_dim,hidden_size,vocab_size):
        '''
        初始化方法
        
        parameter
        --------------
        embedding_dim:int
            解码器词嵌入的维度
        hidden_size:int
            RNN的维度
        output_size:int
            就是词嵌入的数量
        
        '''
        super().__init__()

        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.gru = nn.GRU(embedding_dim,hidden_size,batch_first=True)
        self.out = nn.Linear(hidden_size,vocab_size)

    def forward(self,encoder_outputs,encoder_hidden,target_tensor = None,):
        '''
        parameter
        ----------------
        encoder_output:tensor ,shape = (batch_size,seq_len,hidden_size)
            编码器每个时间步的输出(用于注意力机制)
        encoder_hidden:tensor,shape = (d*num_layer,batch_size,hidden_size)
            编码器的隐藏状态输出,作为解码器的初始隐藏状态
        target_tensor: tensor,shape=(batch_size,target_seq_len)
            目标值，为未经过embeddding的输入数据，用于训练时开启教师强制
        
        return
        ----------------------
        decoder_output:tensor,shape = (batch_size,tager_seq_len,vocab_size) 
            解码器所有时间步的输出层的输出！ 注意是输出层，不是RNN层的输出
        decoder_hidden:tensor ,shape = (d*num_layer,batch_size,tager_seq_len)
        '''
        batch_size = encoder_outputs.size(0)
        # 初始化，解码器的输入，使用SOS——token(开始字符)
        decoder_input = torch.empty(batch_size,1,dtype = torch.long,device=device).fill_(sos_token) # 自己创建的张量，需要指定类型、设备
        # 使用编码器的隐藏状态，初始化解码器的初始隐藏状态
        decoder_hidden = encoder_hidden
        # 用来储存每个时间步的输出
        decoder_outputs = []
        for i in range(max_length):
            decoder_output ,decoder_hidden = self.forward_step(decoder_input,decoder_hidden)
            decoder_outputs.append(decoder_output)
            if target_tensor is not None:
                # 如果使用教师强制，则使用教师强制的输入
                decoder_input = target_tensor[:,i].unsequenze(1)
            else:
                # 如果不使用教师强制，使用当前输出概率最高的词，作为输出
                out = decoder_output.argmax(dim = -1)
                # 每一个输入，不应该与上一个输出相连，否则会有梯度在之间传播，历史就会影响未来
                decoder_input = out.detach() # 使用ditach,断开梯度传播


    def forward_step(self,input,hidden):
        
        pass


